# Example: hycom Data Processing

This notebook outlines some of the basic concepts needed for defining a C3-based achitecture to archive and work with Hycom FMRC data.

## References
https://www.hycom.org/data/gomu0pt04/expt-90pt1m000  
https://www.unidata.ucar.edu/software/tds/current/tutorial/files/FmrcPoster.pdf  
https://tds.hycom.org/thredds/catalog/GOMu0.04/expt_90.1m000/FMRC/runs/catalog.xml

## Background and Goals

The cells below download and access a single run from the hycom simulation for the Gulf of Mexico called "GOMu0.04_901m000_FMRC". FMRC means: Forcast Model Run Collection. The files retrived are in NetCDF format.  [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) is a binary file format (spec/api/library) written on top of the more general [HDF5](https://www.hdfgroup.org/solutions/hdf5/) library.

### Inital Goals
1. Define a type that Mixes `File` and/ or `Client` where Hycom sim data can be collected. We have a initial prototype provisioned called `HycomFMRC`
2. Define a type to handle the data download, possibly mixing the "REST" type.
  - Do file introspection (of NetCDF/HDF5 file) to populate fields of our `HycomFMRC` type once the file is downloaded.
  - Automate retrieval using Cron etc.
3. Explore possibilities for retrieving data from files:
  - One use case: Retrieve a series of 2D slices...over time (say surface temp or something) and be able to either directly load them or stream them.
  
Generally, after solving the storage issue and figuring out source/tranform and entiy types... I am _assuming_ we will want to support the ability to retrive and/or stream data from any one of the datasets(variables) in the collection of runs _across time_.
  
### More on NetCDF
NetCDF files are HDF5 files.  These formats both have rich software ecosystem that support accessing data efficiently and are used to manage large multidimensionall datasets for many large scale HPC-based codes.  IF one were to support the use case I mentioned above using NetCDF/HDF5 only it could be accomplished as follows:
* Create a directory containing the collection of FMRC run files
* Add a "parent" file that contains a dataset that points to  each dataset in the individual run files
* Use the netcdf (or HDF5) library to open the parent file and request an array that does any sort of sliceing and dicing across all the files on desires.


## Requirements
This Notebook requires the py-hycom_1_0_0 kernel.

A prototype `HycomFMRC` type is provision wit hthe `dti-jupyter` package:

In [1]:
from datetime import date
from datetime import timedelta
import xml.etree.ElementTree as ET
import netCDF4 as nc
import requests
import pandas as pd
from pivottablejs import pivot_ui
import xmltodict
from urllib.parse import urlencode,urljoin
import pandas as pd
from IPython.display import display

## Types
The following types are currently provisioned to support Hycom Data:  
(todo: run query to list all Types in hycom- package.)  
```
HycomDataset
HycomFMRC
HycomFMRCFile
GeospatialCoverage
```

In [37]:
#help(c3.HycomDataset)

In [38]:
#help(c3.HycomFMRC)

In [2]:
# Ensure we have a Dataset entry for the desired catalog
cat_url = "https://tds.hycom.org/thredds/catalog/GOMu0.04/expt_90.1m000/FMRC/runs/catalog.xml"
gom_dataset = c3.HycomDataset.upsertHycomDatasetFromCatalog(url = cat_url)

In [18]:
# Grab the HycomDataset record that was created.
objs = c3.HycomDataset.fetch().objs
if objs:
    display(pd.DataFrame(objs.toJson()))


In [4]:
# Create HycomFMRC records for every run that is currenty listed in the catalog
# This uses the...
fmrcs = gom_dataset.upsertFMRCFromDatasetCatalog()
fmrcs

In [17]:
# Grab the HycomFMRC records that were created.
objs = c3.HycomFMRC.fetch().objs
if objs:
    display(pd.DataFrame(objs.toJson()))

In [ ]:
# Detail: look at the timeCoverage for a single HycomFMRC
fmrcs = c3.HycomFMRC.fetch()
fmrcs.objs[0].timeCoverage

In [ ]:
# Download a datafile for each FMRC record
# Note: currently only a fetch of a single timestep is supported, but multiple 
# files can be retrived for a single HycomFMRC record.
# This demo grabs the first available forcast time for the run.
def downloadAll(fmrcs):
    
    fmrc_files = [
        fmr.downloadFMRCRunData(
            time_start = fmr.timeCoverage.start.strftime("%Y-%m-%dT%H:%M:%SZ"),
            time_end = fmr.timeCoverage.start.strftime("%Y-%m-%dT%H:%M:%SZ")
        ) for fmr in fmrcs.objs
    ]
    return fmrc_files
        
downloadAll(fmrcs)

In [15]:
# List the resulting HycomFMRCFile records
objs = c3.HycomFMRCFile.fetch().objs
if objs:
    display(pd.DataFrame(objs.toJson()))

In [14]:
files = c3.FileSystem.inst().listFiles("hycom-data")
files

c3.ListFilesResult()

In [12]:
# ToDoOpen a file to confirm...
# Question: How do I call member functions of type "File" from HycomRMRCFile?
file = c3.HycomFMRCFile.fetch().objs[0]
file.directoryUrl()

500 - NotClassified - c3.love.exceptions.C3RuntimeException_wrapIt [832.42167]
message: "wrapped ClassCastException: c3.type.metadata.impl.PersistableImpl cannot be cast to c3.type.file.ContentMeta"
JSON: {"this": {"type": "HycomFMRCFile", "url": "hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z-2021-08-26T12:00:00Z.nc", "id": "607ca333-1be1-427d-854a-d745580817f5", "name": "GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z-2021-08-26T12:00:00Z.nc", "meta": {"type": "Meta", "tenantTagId": 155, "tenant": "dti-jupyter", "tag": "tc02", "created": "2021-08-30T19:44:46+00:00", "createdBy": "dadams@illinois.edu", "updated": "2021-08-30T19:44:46+00:00", "updatedBy": "dadams@illinois.edu", "timestamp": "2021-08-30T19:44:46+00:00", "fetchInclude": "[]", "fetchType": "HycomFMRCFile"}, "version": 1, "hycomFMRC": {"type": "HycomFMRC", "id": "GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z"}, "timeCoverage": {"type": "TimeRange", "start": "2021-08-26T12:00:00+0

C3RuntimeException: 500 - NotClassified - c3.love.exceptions.C3RuntimeException_wrapIt [832.42167]
message: "wrapped ClassCastException: c3.type.metadata.impl.PersistableImpl cannot be cast to c3.type.file.ContentMeta"
JSON: {"this": {"type": "HycomFMRCFile", "url": "hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z-2021-08-26T12:00:00Z.nc", "id": "607ca333-1be1-427d-854a-d745580817f5", "name": "GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z-2021-08-26T12:00:00Z.nc", "meta": {"type": "Meta", "tenantTagId": 155, "tenant": "dti-jupyter", "tag": "tc02", "created": "2021-08-30T19:44:46+00:00", "createdBy": "dadams@illinois.edu", "updated": "2021-08-30T19:44:46+00:00", "updatedBy": "dadams@illinois.edu", "timestamp": "2021-08-30T19:44:46+00:00", "fetchInclude": "[]", "fetchType": "HycomFMRCFile"}, "version": 1, "hycomFMRC": {"type": "HycomFMRC", "id": "GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z"}, "timeCoverage": {"type": "TimeRange", "start": "2

In [13]:
# Cleanup
print(f"Removed {c3.HycomFMRCFile.removeAll()} HycomFMRCFile records.")
print(f"Removed {c3.HycomFMRC.removeAll()} HycomFMRC records.")
print(f"Removed {c3.HycomDataset.removeAll()} HycomDataset records")
files = c3.FileSystem.inst().listFiles("hycom-data")
if files.files:
    print(f"Deleting {len(files.files)} files")
    c3.FileSystem.inst().deleteFilesBatch(files.files)
print("Done.")

Removed 6 HycomFMRCFile records.
Removed 6 HycomFMRC records.
Removed 1 HycomDataset records
Deleting 6 files


In [5]:
print("https://dataserver3.nccs.nasa.gov/thredds/ncss/bypass/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-23T12:00:00Z?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2021-08-23T12%3A00%3A00Z&time_end=2021-08-29T00%3A00%3A00Z&timeStride=1&vertStride=1&addLatLon=true&accept=netcdf4")

https://dataserver3.nccs.nasa.gov/thredds/ncss/bypass/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-23T12:00:00Z?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2021-08-23T12%3A00%3A00Z&time_end=2021-08-29T00%3A00%3A00Z&timeStride=1&vertStride=1&addLatLon=true&accept=netcdf4


## Manual Download Procedure

In [5]:
from datetime import date
from datetime import timedelta
import xml.etree.ElementTree as ET
import netCDF4 as nc
import requests
import pandas as pd
from pivottablejs import pivot_ui
import xmltodict
from urllib.parse import urlencode,urljoin

In [2]:
def download_ncss_file(url):
    local_filename = url.split('/')[-1].split('?')[0] + '.nc4'
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

def downloadToS3External(srcUrl, fileName, s3_folder):
    tmp_path = "/tmp/" + fileName
    with requests.get(srcUrl, stream=True) as r:
        r.raise_for_status()
        with open(tmp_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    c3.Client.uploadLocalClientFiles(tmp_path, s3_folder, {"peekForMetadata": True})
    #c3.Logger.info("file {} downloaded to {}".format(fileName, s3_folder + fileName))
    os.remove(tmp_path)
    return s3_folder + fileName

In [26]:
url="https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-25T12:00:00Z?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time=2021-08-26T00%3A00%3A00Z&vertCoord=&accept=netcdf4"
downloadToS3External(url,"test01.nc4","hycom-test")

'hycom-testtest01.nc4'

Next steps:
- Add downloadToS3External as a method on HycomFRMC or HycomDataset
- Add method to parse catalog (xml) endpoint to HycmDataset
    - Catalog parsing can result on download calls

In [ ]:
c3.HttpRequest.make(url=)

In [29]:
c3.FileSystem.inst().listFiles("hycom-test")

c3.ListFilesResult(
 files=c3.Arry<File>([c3.AzureFile(
          contentLength=39390145,
          contentLocation='fs/dti/mpodolsky/hycom-test/GOMu0.04_901m000_FMRC_RUN_2021-08-25T12:00:00Z.nc4',
          eTag='"0x8D9699F8AA41780"',
          lastModified=datetime.datetime(2021, 8, 27, 21, 13, 39, tzinfo=datetime.timezone.utc),
          contentMD5='+dG5qmNNa6My21NM8F76OQ==',
          hasMetadata=False,
          url='azure://dev-dti/fs/dti/mpodolsky/hycom-test/GOMu0.04_901m000_FMRC_RUN_2021-08-25T12:00:00Z.nc4',
          blobType='BLOCK_BLOB'),
         c3.AzureFile(
          contentLength=39390145,
          contentLocation='fs/dti/mpodolsky/hycom-test/test01.nc4',
          eTag='"0x8D96A25C61D4F48"',
          lastModified=datetime.datetime(2021, 8, 28, 13, 14, 31, tzinfo=datetime.timezone.utc),
          contentMD5='nEk6ehWNNW302aIrDnB+IA==',
          hasMetadata=False,
          url='azure://dev-dti/fs/dti/mpodolsky/hycom-test/test01.nc4',
          blobType='BLOCK_BLOB')]

In [5]:
print("https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-21T12:00:00Z?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2021-08-21T12%3A00%3A00Z&time_end=2021-08-27T00%3A00%3A00Z&timeStride=1&vertStride=1&addLatLon=true&accept=netcdf4")
def buildHycomFMRCUrl(urlpath,time_start,time_end,
                      vars=['surl_el','salinity','water_temp','water_u','water_v'],
                      disableLLSubset='on',
                      disableProjSubset='on',
                      horizStride=1,
                      timeStride=1,
                      vertStride=1,
                      addLatLon='true',
                      accept='netcdf4'
                     ):
    base_url=f"https://ncss.hycom.org/thredds/ncss/{urlpath}"
    #print(base_url)
    varst = [('var',v) for v in vars]
    url1 = urlencode(varst,{'d':2})
    url2 = urlencode({'disableLLSubset':disableLLSubset,
                      'disableProjSubset':disableProjSubset,
                      'hoizStride':horizStride,
                      'time_start':time_start,
                      'time_end':time_end,
                      'timeStride':timeStride,
                      'vertStride':vertStride,
                      'addLatLon':addLatLon,
                      'accept':accept
                     })
    query = url1+'&'+url2
    url = base_url+'?'+query
    return url

#createHycomDatasetFromCatalog(url="https://tds.hycom.org/thredds/catalog/GOMu0.04/expt_90.1m000/FMRC/runs/catalog.xml")
#ds = getFMRCDataset()

https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-21T12:00:00Z?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2021-08-21T12%3A00%3A00Z&time_end=2021-08-27T00%3A00%3A00Z&timeStride=1&vertStride=1&addLatLon=true&accept=netcdf4


In [7]:
hcds = createHycomDatasetFromCatalog()

[{'run': 'GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z',
  'start': '2021-08-28T12:00:00Z',
  'end': '2021-09-03T00:00:00Z',
  'thredds_url': 'https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z?var=surl_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&hoizStride=1&time_start=2021-08-28T12%3A00%3A00Z&time_end=2021-09-03T00%3A00%3A00Z&timeStride=1&vertStride=1&addLatLon=true&accept=netcdf4'},
 {'run': 'GOMu0.04_901m000_FMRC_RUN_2021-08-27T12:00:00Z',
  'start': '2021-08-27T12:00:00Z',
  'end': '2021-09-02T00:00:00Z',
  'thredds_url': 'https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-27T12:00:00Z?var=surl_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&hoizStride=1&time_start=2021-08-27T12%3A00%3A00Z&time_end=2021-09-02T00%3A00%3A00Z&timeStride=1&vertStride=1&addLatLon=true&acce

### Download a FMRC Run
The following code will download a recent FMRC and print the urls used 
to retrive it along with the header information from the NetCDF file
In particular note the dataset size information:
```
dimensions(sizes): time(1), lat(346), lon(541), depth(40)
```
This means tha tsome variables such as `float32 water_temp(time,depth,lat,lon`) are 4D arrays (3D, really since time = 1) 

In [15]:
today = date.today()
yesterday = today - timedelta(days=1)
daybeforeyesterday = today - timedelta(days=2)

yesterdaystring = yesterday.strftime("%Y-%m-%d") + "T00%3A00%3A00Z"
daybeforeyesterdaystring = daybeforeyesterday.strftime("%Y-%m-%d") + "T12:00:00Z"

url = "https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_" + daybeforeyesterdaystring + "?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time=" + yesterdaystring + "&vertCoord=&accept=netcdf4"
#url = "https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_" + daybeforeyesterdaystring + "?accept=netcdf4"

print(url)
local_filename = download_ncss_file(url)
ds = nc.Dataset(local_filename)
print(ds) 

https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time=2021-08-27T00%3A00%3A00Z&vertCoord=&accept=netcdf4
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    classification_level: UNCLASSIFIED
    distribution_statement: Approved for public release. Distribution unlimited.
    downgrade_date: not applicable
    classification_authority: not applicable
    institution: Naval Oceanographic Office
    source: HYCOM archive file
    history: archv2ncdf3z ;
FMRC Run 2021-08-26T12:00:00Z Dataset
    field_type: instantaneous
    Conventions: CF-1.4, NAVO_netcdf_v1.1
    cdm_data_type: GRID
    featureType: GRID
    location: Proto fmrc:GOMu0.04_901m000_FMRC
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridWriter2)
Original Dataset = fmrc:GOMu0.04_901m

In [51]:
today = date.today()
yesterday = today - timedelta(days=1)
daybeforeyesterday = today - timedelta(days=2)

yesterdaystring = yesterday.strftime("%Y-%m-%d") + "T00%3A00%3A00Z"
daybeforeyesterdaystring = daybeforeyesterday.strftime("%Y-%m-%d") + "T12:00:00Z"

url = "https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_" + daybeforeyesterdaystring + "?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time=" + yesterdaystring + "&vertCoord=&accept=netcdf4"
#url = "https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_" + daybeforeyesterdaystring + "?accept=netcdf4"

#req = c3.HttpRequest(uri=url)
from urllib.parse import urlencode,urljoin

print(url)

print(buildHycomFMRCUrl(run='GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z'))

https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on&disableProjSubset=on&horizStride=1&time=2021-08-27T00%3A00%3A00Z&vertCoord=&accept=netcdf4
https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z
https://ncss.hycom.org/thredds/ncss/GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z?var=surl_el&var=salinity&var=water_temp&var=water_u&var=water_v&disableLLSubset=on


### Access Some Data
Below is a sample cell accessing some of the data.  Note that there are a lot of missing values... for some variables.

In [11]:
time = ds.variables['time'][0]
lat = ds.variables['lat'][:]
lon = ds.variables['lon'][:]
depth = ds.variables['depth'][:]
salinity = ds.variables['salinity'][:]
print(time)
#print(lat)
#print(long)
# A 1D slice for a particular time, depth, and lat:
print(salinity[0,0,0,:])

132.0
[      nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan


/home/c3/.conda/envs/py-hycom_1_0_0/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


In [7]:
#df = pd.DataFrame({'lon': lon})
#pivot_ui(df)

In [3]:
b=False
name = ('x-a' if b else 'y-b')
name

'y-b'

In [53]:
from datetime import date, datetime
from datetime import timedelta
today = datetime.today()
yesterday = today - timedelta(days=1)
daybeforeyesterday = today - timedelta(days=2)

timeRange = {
    "start": today - timedelta(hours=6),
    "end": today
}
timestep = timedelta(hours=1)

timediff = today - daybeforeyesterday

#print(timediff+timestep)

def gentimes():
    timestep = timedelta(hours=1)
    t = timeRange['start']
    while t <= timeRange['end']:
        yield t
        t += timestep

def genranges(times,stepsper):
    st = 0
    en = len(times)
    (start,end) = (times[st],times[st + stepsper-1])
    while st+stepsper-1 <= en:
        yield(start,end)
        st+=1
        (start,end) = (times[st],times[st + stepsper-1])
def genbatches(l,n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]
# file_specs = [
#         {
#             'timeCoverage': {
#                 'start': yesterday,
#                 'end': today
#             }
#         }
#     for 
# ]
print(timeRange)
n=2
times = list(gentimes())
batches = [r for r in genbatches(times,n)]
print (len(times))
print (len(batches))
print(batches[0][0],batches[0][len(batches[0])-1])
#times = gentimes()
#print(next(times))
#print(next(times))
spec = [
    {
    'start': batches[n][0],
    'end': batches[n][-1]
}
    for n in range(len(batches))
]
print(spec)
# times = range()

# times = [
#     timeRange['start'] + dt
#     while timestep
# ]

yesterdaystring = yesterday.strftime("%Y-%m-%d") + "T00%3A00%3A00Z"
daybeforeyesterdaystring = daybeforeyesterday.strftime("%Y-%m-%d") + "T12:00:00Z"

{'start': datetime.datetime(2021, 9, 2, 8, 3, 24, 942660), 'end': datetime.datetime(2021, 9, 2, 14, 3, 24, 942660)}
7
4
2021-09-02 08:03:24.942660 2021-09-02 09:03:24.942660
[{'start': datetime.datetime(2021, 9, 2, 8, 3, 24, 942660), 'end': datetime.datetime(2021, 9, 2, 9, 3, 24, 942660)}, {'start': datetime.datetime(2021, 9, 2, 10, 3, 24, 942660), 'end': datetime.datetime(2021, 9, 2, 11, 3, 24, 942660)}, {'start': datetime.datetime(2021, 9, 2, 12, 3, 24, 942660), 'end': datetime.datetime(2021, 9, 2, 13, 3, 24, 942660)}, {'start': datetime.datetime(2021, 9, 2, 14, 3, 24, 942660), 'end': datetime.datetime(2021, 9, 2, 14, 3, 24, 942660)}]
